<a href="https://colab.research.google.com/github/AhamadAli30/Knee-Osteoarthritis/blob/main/Prediction_with_Sequential_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ( '/content/drive/' )

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#!unzip -uq  "/content/drive/MyDrive/OA MEDICAL ANALYSIS/Knee OA Dataset-20221224T072217Z-001.zip" -d "/content/drive/MyDrive/OA MEDICAL ANALYSIS/Knee OA Dataset"

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Dropout , Flatten , Dense
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.losses
from tensorflow.keras.preprocessing.image import DirectoryIterator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
main_dir="/content/drive/MyDrive/OA medical "
train_dir=os.path.join ( main_dir , 'Training Data' )
test_dir=os.path.join ( main_dir , 'Testing Data' )
valid_dir=os.path.join ( main_dir , 'Validation Data'  )

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=datagen.flow_from_directory(directory=train_dir,
                                                  target_size=(150,150),
                                                  class_mode="sparse",
                                                  batch_size=32 )

test_generator=datagen.flow_from_directory(directory=test_dir,
                                                target_size=(150,150),
                                                class_mode='sparse',
                                                batch_size=32 )

validation_generator=datagen.flow_from_directory(directory=valid_dir,
                                                target_size=(150,150),
                                                class_mode='sparse',
                                                batch_size=32 )


Found 97 images belonging to 4 classes.
Found 0 images belonging to 0 classes.
Found 83 images belonging to 4 classes.


In [ ]:
train_generator.class_indices

{'Mild OA': 0, 'Moderate OA': 1, 'Normal Knee': 2, 'Severe OA': 3}

In [ ]:
train_generator.image_shape

(150, 150, 3)

In [ ]:
# define the model architecture
model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(128, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(4, activation='softmax')
])





In [ ]:
#model.compile(loss='categorical_hinge', optimizer='adam', metrics=['accuracy'])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
nb_train_samples=97
nb_validation_samples=83
batch_size=32
epoch=50

In [ ]:
history = model.fit(train_generator,epochs=50,validation_data=validation_generator)

Epoch 1/50
4/4 [==============================] - 9s 2s/step - loss: 1.7396 - accuracy: 0.3093 - val_loss: 1.4131 - val_accuracy: 0.3373
Epoch 2/50
4/4 [==============================] - 7s 2s/step - loss: 1.3937 - accuracy: 0.3196 - val_loss: 1.3320 - val_accuracy: 0.3494
Epoch 3/50
4/4 [==============================] - 9s 2s/step - loss: 1.3188 - accuracy: 0.3711 - val_loss: 1.2983 - val_accuracy: 0.3494
Epoch 4/50
4/4 [==============================] - 7s 2s/step - loss: 1.3967 - accuracy: 0.3505 - val_loss: 1.4614 - val_accuracy: 0.3373
Epoch 5/50
4/4 [==============================] - 7s 2s/step - loss: 1.3873 - accuracy: 0.3093 - val_loss: 1.3043 - val_accuracy: 0.3373
Epoch 6/50
4/4 [==============================] - 7s 2s/step - loss: 1.2997 - accuracy: 0.3402 - val_loss: 1.3104 - val_accuracy: 0.3494
Epoch 7/50
4/4 [==============================] - 7s 2s/step - loss: 1.2963 - accuracy: 0.3711 - val_loss: 1.2966 - val_accuracy: 0.3494
Epoch 8/50
4/4 [=========================

In [ ]:
test_loss,test_acc=model.evaluate(validation_generator)
print('test acc:{}test loss: {}'.format(test_acc,test_loss))

3/3 [==============================] - 2s 517ms/step - loss: 4.7092 - accuracy: 0.2048
test acc:0.20481927692890167test loss: 4.709201335906982


In [ ]:
import numpy as np
from google.colab import files
from tensorflow import keras
from keras.preprocessing import image

import cv2

uploaded=files.upload()
for f in uploaded.keys():
  img_path="/content/"+f
  img=keras.preprocessing.image.load_img(img_path, target_size=(150,150))
  images =keras.preprocessing.image.img_to_array ( img )
  images = np.expand_dims ( images , axis = 0 )
  prediction=model.predict(images)
  if prediction.all()==1:
    print(f," it is moderate OA")

  elif prediction.any()==2:
    print(f," it is Normal OA")
  elif prediction.all()==0:
    print(f," it is mild OA")
  elif prediction.any()==3:
    print(f," it is severe OA")

Saving Patient5_Axial.jpeg to Patient5_Axial (3).jpeg
1/1 [==============================] - 0s 32ms/step
Patient5_Axial.jpeg  it is mild OA


In [ ]:
model.save('medical_OA_model.h5')